In [127]:
import sagemaker
sess=sagemaker.Session(default_bucket="sagemaker-us-east-1-470086202700")
role= sagemaker.get_execution_role()
account =sess.boto_session.client('sts').get_caller_identity()['Account']
region= sess.boto_session.region_name
role

'arn:aws:iam::470086202700:role/service-role/AmazonSageMaker-ExecutionRole-20200917T091404'

In [185]:
# change names of below to project name and image name
repo_name="dyco3d"
image_tag=repo_name
base_job_name=image_tag

In [186]:
### Set Enivronment variables

In [187]:
%env dockerfile sagemaker-DyCo3D/Docker/Dockerfile
%env account {account}
%env region {region}
%env repo_name {repo_name}
%env image_tag {image_tag}

env: dockerfile=sagemaker-DyCo3D/Docker/Dockerfile
env: account=470086202700
env: region=us-east-1
env: repo_name=dyco3d
env: image_tag=dyco3d


In [195]:
!cat $dockerfile

#FROM nvidia/cuda:9.0-cudnn7-devel-ubuntu16.04
#FROM nvidia/cuda:9.2-cudnn7-devel-ubuntu18.04
FROM nvidia/cuda:10.2-cudnn7-devel-ubuntu18.04

# Install system dependencies
#ENV DEBIAN_FRONTEND=noninteractive 
RUN apt-get update 
RUN  apt-get install -y wget build-essential git
RUN ln -s /usr/lib/x86_64-linux-gnu/libmpfr.so.6 /usr/lib/x86_64-linux-gnu/libmpfr.so.4

# Install python dependencies
ENV PYTHON_VERSION=3.7
ENV LANG=C.UTF-8 LC_ALL=C.UTF-8
ENV PATH /opt/conda/bin:$PATH

RUN wget --quiet https://repo.anaconda.com/miniconda/Miniconda3-4.5.12-Linux-x86_64.sh -O ~/miniconda.sh && \
    /bin/bash ~/miniconda.sh -b -p /opt/conda && \
    rm ~/miniconda.sh && \
	conda install -y -c bioconda google-sparsehash && \
	conda install -y libboost && \ 
	conda install -y -c daleydeng gcc-5
    #conda install -y scipy pillow gunicorn==19.9.0 && \
    #conda install -y gevent flask matplotlib && \
    #conda install -y -c conda-forge opencv

RUN conda install -y scipy

ENV PYTHONUNBUFFERED=TRUE

In [196]:
# Build Docker
!docker build -t $image_tag -f $dockerfile .

Sending build context to Docker daemon  390.6MB
Step 1/27 : FROM nvidia/cuda:10.2-cudnn7-devel-ubuntu18.04
 ---> 301c8f9f1c90
Step 2/27 : RUN apt-get update
 ---> Using cache
 ---> 2d0c35950295
Step 3/27 : RUN  apt-get install -y wget build-essential git
 ---> Using cache
 ---> 45a8faa3cc10
Step 4/27 : RUN ln -s /usr/lib/x86_64-linux-gnu/libmpfr.so.6 /usr/lib/x86_64-linux-gnu/libmpfr.so.4
 ---> Using cache
 ---> 1892ea96bd01
Step 5/27 : ENV PYTHON_VERSION=3.7
 ---> Using cache
 ---> e5a35dc2f0d3
Step 6/27 : ENV LANG=C.UTF-8 LC_ALL=C.UTF-8
 ---> Using cache
 ---> 199ff84be113
Step 7/27 : ENV PATH /opt/conda/bin:$PATH
 ---> Using cache
 ---> 2b7d44910684
Step 8/27 : RUN wget --quiet https://repo.anaconda.com/miniconda/Miniconda3-4.5.12-Linux-x86_64.sh -O ~/miniconda.sh &&     /bin/bash ~/miniconda.sh -b -p /opt/conda &&     rm ~/miniconda.sh && 	conda install -y -c bioconda google-sparsehash && 	conda install -y libboost && 	conda install -y -c daleydeng gcc-5
 ---> Using cache
 ---> 0bf

In [197]:
 !docker images

REPOSITORY                                            TAG                             IMAGE ID       CREATED                  SIZE
dyco3d                                                latest                          8c3bd5031c53   Less than a second ago   9.06GB
470086202700.dkr.ecr.us-east-1.amazonaws.com/dyco3d   latest                          bbb6b72f01ac   21 minutes ago           9.06GB
470086202700.dkr.ecr.us-east-1.amazonaws.com/dyco3d   <none>                          0c6e824d06a6   5 days ago               9.06GB
470086202700.dkr.ecr.us-east-1.amazonaws.com/dyco3d   <none>                          af75e93c62eb   5 days ago               9.06GB
470086202700.dkr.ecr.us-east-1.amazonaws.com/dyco3d   <none>                          04de36a4ab8a   5 days ago               9.06GB
470086202700.dkr.ecr.us-east-1.amazonaws.com/dyco3d   <none>                          5a3e4db56081   5 days ago               9.06GB
470086202700.dkr.ecr.us-east-1.amazonaws.com/dyco3d   <none>           

In [198]:
!docker tag $image_tag  $account.dkr.ecr.$region.amazonaws.com/$repo_name:latest

In [199]:
## Below cells sets up enironment variables build dokcer image and push to ECR If the image is already in ECR skip these steps

In [200]:
%%sh
aws ecr describe-repositories --repository-names $repo_name > /dev/null 2>&1
if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name $repo_name > /dev/null
fi

$(aws ecr get-login --region $region --no-include-email)

Login Succeeded


WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



In [201]:
 !docker push $account.dkr.ecr.$region.amazonaws.com/$repo_name:latest

The push refers to repository [470086202700.dkr.ecr.us-east-1.amazonaws.com/dyco3d]

8d2e7d88: Preparing 
c82a5f19: Preparing 
1fd91636: Preparing 
a02348d5: Preparing 
cc24ce8b: Preparing 
8ebd2af2: Preparing 
f36ba3b1: Preparing 
7702791b: Preparing 
3a0bc4ea: Preparing 
d3fdb5ae: Preparing 
4d008330: Preparing 
83d94027: Preparing 
aad62914: Waiting g 
de2a7278: Preparing 
de2a7278: Waiting g 
52f0cf56: Preparing 
56eb5bc2: Preparing 
4493533f: Preparing 
dd550fb3: Preparing 
2e9f2712: Preparing 
cfab2a57: Preparing 
54cbc197: Waiting g 
6a5f5aba: Preparing 
d64dc157: Preparing 
1313d984: Preparing 
d2e7d88: Pushed   98.14MB/98.09MB1Alatest: digest: sha256:d5d31fcbc0bca6340e89ca0aba0bd929067cad94c04708558d558d8ccb60ba7d size: 5797
